# Difference between two netcdf files

This is a short notebook that will allow you to compare the values between two netcdf files and will produce a netcdf file with the differences as well as a small figure.

## Instructions
1. Upload the 2 data files to the current directory
1. Change global variables to desired values
1. Run all cells

## Set variables

In [ ]:
# Input files
FILENAME1 = os.path.join(".", "FILENAME_HERE.nc")
FILENAME2 = os.path.join(".", "FILENAME_HERE.nc")

# Outputfile

OUTPUT_FILE = os.path.join(".", "differences.nc")

## Necessary imports

In [ ]:
# Os manipulations mainly path
import os
# NetCDF loading and manip
import xarray as xr

# Adding holoviz support to xarray
import hvplot.xarray

# Basic matplotlib graphing
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from matplotlib.colors import LinearSegmentedColormap

## Some custom colormaps

In [ ]:
fzcmap = LinearSegmentedColormap.from_list("my_colormap", ((0,0.2,0.4),(0,0.2,0.4),'slateblue','deepskyblue','aqua','antiquewhite','yellow','darkorange','red',(0.54,0,0),(0.54,0,0)), N=256, gamma=1.0)
blackcmap = LinearSegmentedColormap.from_list("my_colormap", ('k', 'k'), N=256, gamma=1.0)
landseacmap = LinearSegmentedColormap.from_list("my_colormap", ((0.22,0.22,0.56),(0.38,0.49,0.99),(0.51,0.99,1),(0.01,0.51,0),(0.7,0.7,0)), N=256, gamma=1.0)
bathycmap = LinearSegmentedColormap.from_list("my_colormap", ((0.1601562, 0.28125, 0.703125),(0.28125, 0.484375, 0.859375),(0.8671875, 0.9101562, 0.9804688)), N=256, gamma=1.0)

## Load the data

In [ ]:
ds1 = xr.open_dataset(FILENAME1)

ds2 = xr.open_dataset(FILENAME2)

## Quick Data overview

In [ ]:
ds1

In [ ]:
ds2

## Simple plotting

In [ ]:
ds1.TPP_ALLK.hvplot.image(cmap=fzcmap)

In [ ]:
ds2.TPP_ALLK.hvplot.image()

## Calculate the difference

In [ ]:
# The TIME_COUNTER are different for each dataset but as we only have one array we will drop the values
difference = (ds1.drop('TIME_COUNTER').TPP_ALLK - ds2.drop('TIME_COUNTER').TPP_ALLK)
difference.hvplot()

## Write the data to disk

In [ ]:
difference.to_netcdf(OUTPUT_FILE)

## Make a standard plot

In [ ]:
plt.figure(figsize=(6, 4.5))
ax = plt.axes(projection=ccrs.Robinson())
# ax.gridlines()
cf = difference.plot(transform=ccrs.PlateCarree(), ax=ax, cmap=fzcmap, add_colorbar=False);
# ax.set(aspect=2)
# ax.set_aspect('auto')
ax.tick_params(labelsize=15)
# ax.set_title('Plot Anomaly', weight='normal', fontsize=15)
# plt.title('Plot Anomaly', weight='normal', fontsize=15);

gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=False,
              linewidth=.5, color='k', alpha=1., linestyle='-', zorder=999)
gl.xlabels_top = False
gl.ylabels_left = False
gl.xlines = True
gl.xlocator = mticker.FixedLocator([-180, -120, -60, 0, 60, 120, 180])
gl.xformatter = LONGITUDE_FORMATTER
gl.ylocator = mticker.FixedLocator([-90, -60, -30, 0, 30, 60, 90])
gl.yformatter = LATITUDE_FORMATTER
gl.xlabel_style = {'size': 15, 'color': 'gray'}
gl.xlabel_style = {'color': 'red', 'weight': 'bold'}
cb = plt.colorbar(cf, orientation='horizontal',extend='both')
cf.cmap.set_under('darkblue')
cf.cmap.set_over('sienna')
cb.ax.tick_params(labelsize=15)
cb.ax.set_title('Prod Anomaly', weight='normal', fontsize=15)
ax.outline_patch.set_linewidth(1.6)
ax.outline_patch.set_zorder(3000)
# plt.tight_layout()
# plt.savefig(fname,format='png', dpi=450)
# ax.outline_patch.set_linewidth(1.6)
# ax.outline_patch.set_zorder(3000)



plt.savefig('anomaly.tif', dpi=450)